In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, accuracy_score

# ==== 1. CONFIGURATION ====
data_dir = r"E:\Collaboration Work\With Farooq\Bearings MDPI\CWT_Log\For Channel-02"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_epochs = 15
num_classes = 4  # Inner, Outer, Roller, Normal
img_size = 128
patch_size = 16

# ==== 2. TRANSFORMS & DATA LOADING ====
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes
print(f"Classes: {class_names}")

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# ==== 3. TRANSFORMER MODEL ====
class SimpleViT(nn.Module):
    def __init__(self, img_size, patch_size, num_classes, dim=128, depth=6, heads=8, mlp_dim=256):
        super().__init__()
        num_patches = (img_size // patch_size) ** 2
        patch_dim = patch_size * patch_size

        self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)
        self.linear_proj = nn.Linear(patch_dim, dim)

        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.unfold(x).transpose(1, 2)  # Shape: (B, num_patches, patch_dim)
        x = self.linear_proj(x)  # Shape: (B, num_patches, dim)
        cls_token = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x = x + self.pos_embedding[:, :x.size(1)]
        x = self.transformer(x)
        return self.mlp_head(x[:, 0])

model = SimpleViT(img_size=img_size, patch_size=patch_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ==== 4. TRAINING LOOP ====
for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

# ==== 5. EVALUATION ====
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

acc = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=class_names)

print("\n=== Final Results ===")
print(f"Validation Accuracy: {acc * 100:.2f}%")
print(report)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, accuracy_score

# ==== Configuration ====
data_dir = r"E:\Collaboration Work\With Farooq\Bearings MDPI\CWT_Log\For Channel-02"
batch_size = 32
num_epochs = 3
learning_rate = 0.0001
img_size = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Transforms ====
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# ==== Load full dataset ====
full_dataset = datasets.ImageFolder(data_dir, transform=transform)
targets = [label for _, label in full_dataset.samples]

# ==== Stratified Split ====
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, val_idx in splitter.split(torch.zeros(len(targets)), targets):
    train_dataset = Subset(full_dataset, train_idx)
    val_dataset = Subset(full_dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"✅ Classes: {full_dataset.classes}")

# ==== Vision Transformer ====
model = models.vit_b_16(pretrained=True)
in_features = model.heads[0].in_features
model.heads = nn.Sequential(nn.Linear(in_features, len(full_dataset.classes)))
model = model.to(device)

# ==== Loss, Optimizer, Scheduler ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# ==== Training Function ====
def train_model():
    model.train()
    for epoch in range(num_epochs):
        total_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        scheduler.step()
        train_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss:.4f}, Train Acc: {train_acc:.2f}%")

# ==== Evaluation Function ====
def evaluate_model():
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"\n✅ Final Validation Accuracy: {acc * 100:.2f}%")
    print("\n=== Classification Report ===")
    print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))

# ==== Run ====
train_model()
evaluate_model()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import numpy as np

# ==== Extract Features for t-SNE and Confusion Matrix ====
model.eval()
all_features = []
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Extract penultimate features
        features = model._modules['heads'][0].in_features
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        # Get embeddings from penultimate layer
        hook_output = []
        def hook(module, input, output):
            hook_output.append(output.cpu().numpy())
        
        handle = model.heads[0].register_forward_hook(hook)
        _ = model(images)
        handle.remove()
        embeddings = hook_output[0]
        
        all_features.append(embeddings)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ==== Prepare Data ====
features_np = np.concatenate(all_features, axis=0)
labels_np = np.array(all_labels)

# ==== t-SNE Plot ====
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
features_tsne = tsne.fit_transform(features_np)

plt.figure(figsize=(10, 7))
palette = sns.color_palette("hls", len(full_dataset.classes))
sns.scatterplot(x=features_tsne[:, 0], y=features_tsne[:, 1], hue=labels_np, palette=palette, legend='full', alpha=0.7)
plt.title("t-SNE Visualization of Extracted Features", fontsize=14, fontweight='bold')
plt.xlabel("Dimension 1", fontsize=12)
plt.ylabel("Dimension 2", fontsize=12)
plt.legend(title="Class", labels=full_dataset.classes)
plt.grid(True)
plt.tight_layout()
plt.show()

# ==== Confusion Matrix ====
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=full_dataset.classes)
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap="Blues", colorbar=True)
plt.title("Confusion Matrix", fontsize=14, fontweight='bold')
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 7))
palette = sns.color_palette("hls", len(full_dataset.classes))
sns.scatterplot(x=features_tsne[:, 0], y=features_tsne[:, 1], hue=labels_np, palette=palette, legend='full', alpha=0.7)
#lt.title("t-SNE Visualization of Extracted Features", fontsize=14, fontweight='bold')
plt.xlabel("Dimension 1", fontsize=12)
plt.ylabel("Dimension 2", fontsize=12)
plt.legend(title="Class", labels=full_dataset.classes)
#plt.grid(True)
plt.tight_layout()
plt.show()